In [64]:
# Holden Higgins
# 9-25-22

In [133]:
import json
import urllib3
import time
from datetime import datetime

In [167]:
api_key = input('Paste your api key')

Paste your api key 


In [161]:
host = "https://api.the-odds-api.com"
oddsFormat = "decimal"
regions = "us"
sports_endpoint = f"{host}/v4/sports/?apiKey={api_key}"

In [162]:
IL_books = {
    'betmgm',
    'barstool',
    'pointsbetus',
    'williamhill_us',
    'fanduel',
    'draftkings',
    'betrivers',
    'unibet'}

In [163]:
valid_sports = [
    'americanfootball_ncaaf',
    'americanfootball_nfl',
    'baseball_mlb',
    'basketball_nba',
    'cricket_caribbean_premier_league',
    'cricket_international_t20',
    'icehockey_nhl',
    'icehockey_sweden_allsvenskan',
    'icehockey_sweden_hockey_league',
    'mma_mixed_martial_arts',
    'rugbyleague_nrl',
    'soccer_argentina_primera_division',
    'soccer_australia_aleague',
    'soccer_belgium_first_div',
    'soccer_brazil_campeonato',
    'soccer_brazil_serie_b',
    'soccer_chile_campeonato',
    'soccer_china_superleague',
    'soccer_conmebol_copa_libertadores',
    'soccer_denmark_superliga',
    'soccer_england_league1',
    'soccer_england_league2',
    'soccer_epl',
    'soccer_finland_veikkausliiga',
    'soccer_france_ligue_one',
    'soccer_france_ligue_two',
    'soccer_germany_bundesliga',
    'soccer_germany_bundesliga2',
    'soccer_italy_serie_a',
    'soccer_italy_serie_b',
    'soccer_japan_j_league',
    'soccer_league_of_ireland',
    'soccer_mexico_ligamx',
    'soccer_netherlands_eredivisie',
    'soccer_norway_eliteserien',
    'soccer_poland_ekstraklasa',
    'soccer_portugal_primeira_liga',
    'soccer_spain_la_liga',
    'soccer_spain_segunda_division',
    'soccer_spl',
    'soccer_sweden_allsvenskan',
    'soccer_sweden_superettan',
    'soccer_switzerland_superleague',
    'soccer_turkey_super_league',
    'soccer_uefa_champs_league',
    'soccer_uefa_europa_league',
    'soccer_uefa_nations_league',
    'soccer_usa_mls']

In [164]:
http = urllib3.PoolManager()

In [165]:
r = http.request('GET', sports_endpoint)
r.status

200

In [166]:
sports = [sport["key"] for sport in json.loads(r.data)]

In [72]:
# BE careful! this uses api keys
data_by_sport = {}
for sport in valid_sports:
    single_sport_endpoint = f"{host}/v4/sports/{sport}/odds?regions={regions}&oddsFormat={oddsFormat}&apiKey={api_key}"
    r = http.request("GET", single_sport_endpoint)
    data_by_sport[sport] = json.loads(r.data)
    print(f"fetched {sport}")
    time.sleep(1)

fetched americanfootball_ncaaf
fetched americanfootball_nfl
fetched baseball_mlb
fetched basketball_nba
fetched cricket_caribbean_premier_league
fetched cricket_icc_world_cup
fetched cricket_international_t20
fetched icehockey_nhl
fetched icehockey_sweden_allsvenskan
fetched icehockey_sweden_hockey_league
fetched mma_mixed_martial_arts
fetched rugbyleague_nrl
fetched soccer_argentina_primera_division
fetched soccer_australia_aleague
fetched soccer_belgium_first_div
fetched soccer_brazil_campeonato
fetched soccer_brazil_serie_b
fetched soccer_chile_campeonato
fetched soccer_china_superleague
fetched soccer_conmebol_copa_libertadores
fetched soccer_denmark_superliga
fetched soccer_england_league1
fetched soccer_england_league2
fetched soccer_epl
fetched soccer_fifa_world_cup
fetched soccer_finland_veikkausliiga
fetched soccer_france_ligue_one
fetched soccer_france_ligue_two
fetched soccer_germany_bundesliga
fetched soccer_germany_bundesliga2
fetched soccer_italy_serie_a
fetched soccer_it

In [168]:
r.headers

HTTPHeaderDict({'Date': 'Sun, 25 Sep 2022 23:32:06 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '9028', 'Connection': 'keep-alive', 'vary': 'Accept-Encoding', 'X-Requests-Used': '53', 'X-Requests-Remaining': '447', 'Apigw-Requestid': 'ZCi2mi9hoAMESiQ='})

In [76]:
# [(sport, [book['key'] for book in sport_data[0]['bookmakers']]) for sport, sport_data in data_by_sport.items()]

In [151]:
def get_pos_ev_bets(sport_data, can_draw):
    for game in sport_data:
        if datetime.fromisoformat(game['commence_time'].strip('Z')) < datetime.utcnow():
            continue
        home_team = game['home_team']
        away_team = game['away_team']
        game_odds_by_bookmaker = {}
        for bookmaker in game['bookmakers']:
            if bookmaker['key'] not in IL_books:
                continue
            bookmaker_odds = bookmaker['markets'][0]
            game_odds_by_bookmaker[bookmaker['key']] = {outcome['name']:outcome['price'] for outcome in bookmaker_odds['outcomes']}

        if can_draw:
            for bookmaker, game_odds in game_odds_by_bookmaker.items():
                for bookmaker2, game_odds2 in game_odds_by_bookmaker.items():
                    for bookmaker3, game_odds3 in game_odds_by_bookmaker.items():
                        implied_probability = 1/game_odds[home_team] + 1/game_odds2[away_team] + 1/game_odds3['Draw']
                        if implied_probability < 1:
                            print("{:.3f}".format(implied_probability), bookmaker, bookmaker2, bookmaker3, f"{home_team} vs. {away_team}")
        else:
            for bookmaker, game_odds in game_odds_by_bookmaker.items():
                for bookmaker2, game_odds2 in game_odds_by_bookmaker.items():
                    implied_probability = 1/game_odds[home_team] + 1/game_odds2[away_team]
                    if implied_probability < 1:
                        print("{:.3f}".format(implied_probability), bookmaker, bookmaker2, f"{home_team} vs. {away_team}")

In [153]:
for sport, sport_data in data_by_sport.items():
    print(sport + "==========================================")
    can_draw = sport.startswith("soccer") or sport.startswith("icehockey_sweden")
    get_pos_ev_bets(sport_data, can_draw)
    
    

americanfootball_ncaaf==========================================
americanfootball_nfl==========================================
baseball_mlb==========================================
basketball_nba==========================================
0.998 fanduel unibet Los Angeles Lakers vs. Los Angeles Clippers
0.998 fanduel barstool Los Angeles Lakers vs. Los Angeles Clippers
0.998 fanduel betrivers Los Angeles Lakers vs. Los Angeles Clippers
cricket_caribbean_premier_league==========================================
cricket_icc_world_cup==========================================
cricket_international_t20==========================================
icehockey_nhl==========================================
icehockey_sweden_allsvenskan==========================================
icehockey_sweden_hockey_league==========================================
mma_mixed_martial_arts==========================================
rugbyleague_nrl==========================================
soccer_argentina_primera_divis